# Multi-Objective Bayesian Optimization (MOBO)

This notebook loads experimental results, fits a multi-output GP using BoTorch, and proposes new candidates via qEHVI.

Requirements:
- `src/model.py`
- `src/utils.py`
- `data/raw/batch0_results.csv`

In [ ]:
import pandas as pd
import torch
from src.utils import x_normalizer
from src.model import fit_gp_models
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.optim import optimize_acqf
from botorch.sampling import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions import NondominatedPartitioning

# Set precision
torch.set_default_dtype(torch.float64)

# Load and normalize data
df = pd.read_csv("../data/raw/batch0_results.csv")
X = df.iloc[:, 0:8].values
Y = df[['PCE', 'Stability', 'Repeatability']].values

var_array = [X[:, i] for i in range(X.shape[1])]
X_norm = torch.tensor(x_normalizer(X, var_array))
Y_tensor = torch.tensor(Y, dtype=torch.float64)
Y_scaled = (Y_tensor - Y_tensor.min(dim=0).values) / (Y_tensor.max(dim=0).values - Y_tensor.min(dim=0).values)

# Fit model
model = fit_gp_models(X_norm, Y_scaled)

# Define acquisition function
ref_point = Y_scaled.min(dim=0).values - 0.01
partitioning = NondominatedPartitioning(ref_point=ref_point, Y=Y_scaled)
acq = qExpectedHypervolumeImprovement(model=model, ref_point=ref_point, partitioning=partitioning, sampler=SobolQMCNormalSampler(num_samples=128))

# Optimize acquisition
bounds = torch.stack([torch.zeros(8), torch.ones(8)])
candidates, _ = optimize_acqf(acq_function=acq, bounds=bounds, q=1, num_restarts=10, raw_samples=100)

# Denormalize and snap to grid
candidates_np = candidates.detach().numpy()
suggested = get_closest_array(candidates_np, var_array)

pd.DataFrame(suggested, columns=df.columns[:8])